In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load data
data = pd.read_csv('Data-Copy1.csv')
if "Unnamed: 0" in data.columns:
    data.drop(columns=["Unnamed: 0"], inplace=True)

# Define semester labels
semester_labels = [f"SEM-{i}" for i in range(1, 9)]

# Feature Engineering
data['Avg_SGPA'] = data.iloc[:, :-2].mean(axis=1)
data['SGPA_Improvement'] = data.iloc[:, 1:-2].subtract(data.iloc[:, :-3].values).mean(axis=1)
data['SGPA_Difference'] = data.iloc[:, -3] - data.iloc[:, 0]

# Initialize dictionaries
x_train_test = {}
y_train_test = {}
scalers = {}
mlp_models = {}

# Ask if user wants performance evaluation
evaluate_performance = input("Do you want to evaluate model performance? (yes/no): ").lower().strip() == 'yes'

# Evaluation function
def evaluate_model(model, x_test_scaled, y_test, sem_label):
    y_pred = model.predict(x_test_scaled)
    print(f"Model Performance for {sem_label}:")
    print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
    print(f"R² Score: {model.score(x_test_scaled, y_test):.4f}\n")

# Train and save models for each semester
for i in range(2, 9):
    predictors = semester_labels[: i - 1] + ["Avg_SGPA", "SGPA_Improvement", "SGPA_Difference"]
    target = f"SEM-{i}"

    x = data[predictors]
    y = data[target]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)
    
    # Store train/test sets if evaluation is needed
    if evaluate_performance:
        x_train_test[target] = (x_train, x_test)
        y_train_test[target] = (y_train, y_test)

    # Standardization
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    scalers[target] = scaler

    # Train model
    model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', 
                         max_iter=1000, random_state=2, early_stopping=True)
    model.fit(x_train_scaled, y_train)
    mlp_models[target] = model

    # Save model and scaler
    joblib.dump(model, f"{target}_model.pkl")
    joblib.dump(scaler, f"{target}_scaler.pkl")
    
    # Evaluate model if requested
    if evaluate_performance:
        evaluate_model(model, x_test_scaled, y_test, target)

print("Models trained and saved successfully!")